In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [ ]:
# Define dataset path
BASE_DIR = '../dataset/emotion-recog-dataset'

# Emotion labels
emotion_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [ ]:
# Prepare image paths and labels
image_paths = []
emotion_classes = []

for emotion in emotion_labels:
    emotion_dir = os.path.join(BASE_DIR, emotion)
    for filename in tqdm(os.listdir(emotion_dir)):
        image_path = os.path.join(emotion_dir, filename)
        image_paths.append(image_path)
        emotion_classes.append(emotion_labels.index(emotion))

In [ ]:
# Convert to DataFrame
df = pd.DataFrame({
    'image': image_paths,
    'emotion': emotion_classes
})

In [ ]:
# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Display the first few rows of the DataFrame
print(df.columns)
print(df.head())

In [ ]:
# Map emotion labels to their names
emotion_dict = {i: emotion for i, emotion in enumerate(emotion_labels)}

In [ ]:
# Display the first image with its label
img = Image.open(df['image'][0])
plt.axis('off')
plt.imshow(img)
plt.title(f"Emotion: {emotion_dict[df['emotion'][0]]}")
plt.show()

In [ ]:
# Plot the distribution of emotions
sns.countplot(x='emotion', data=df, palette='Set2')
plt.title("Emotion Distribution")
plt.show()

In [ ]:
# Display a grid of images with labels
plt.figure(figsize=(20, 20))
files = df.iloc[0:25]

In [ ]:
plt.figure(figsize=(20, 20))

# Adjust the spacing between images and titles
plt.subplots_adjust(hspace=0.4, wspace=0.2)  # hspace controls height space, wspace controls width space

for index, row in enumerate(files.itertuples(index=False, name=None), 1):
    plt.subplot(5, 5, index)
    
    file = row[0]
    emotion = row[1]
    
    img = load_img(file)
    img = np.array(img)
    plt.imshow(img)
    plt.title(f"Emotion: {emotion_dict[emotion]}")
    plt.axis('off')

plt.show()

In [ ]:
# Function to extract features from images
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = img.resize((128, 128), Image.Resampling.LANCZOS) 
        img = np.array(img)
        features.append(img)

    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)
    return features

In [ ]:
X = extract_features(df['image'])
X.shape

In [ ]:
# Normalize the images
X = X / 255.0

In [ ]:
# Convert labels to one-hot encoding
y_emotion = to_categorical(df['emotion'], num_classes=len(emotion_labels))

In [ ]:
# Define the model input shape
input_shape = (128, 128, 1)
inputs = Input((input_shape))

In [ ]:
# Define the convolutional layers
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
dropout_conv_1 = Dropout(0.5)(maxp_1)  # Add dropout in conv layer

conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(dropout_conv_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
dropout_conv_2 = Dropout(0.5)(maxp_2)  # Add dropout in conv layer

conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(dropout_conv_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)
dropout_conv_3 = Dropout(0.5)(maxp_3)  # Add dropout in conv layer

conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu')(dropout_conv_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)
dropout_conv_4 = Dropout(0.5)(maxp_4)  # Add dropout in conv layer

flatten = Flatten()(dropout_conv_4)

# Fully connected layers with L2 regularization
dense_1 = Dense(256, activation='relu', kernel_regularizer=l2(0.02))(flatten)
dense_2 = Dense(256, activation='relu', kernel_regularizer=l2(0.02))(flatten)

dropout_1 = Dropout(0.6)(dense_1)
dropout_2 = Dropout(0.6)(dense_2)

# Output layer for emotion classification
output_1 = Dense(len(emotion_labels), activation='softmax', name='emotion_out')(dropout_2)

model = Model(inputs=[inputs], outputs=[output_1])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
    ModelCheckpoint('../models/best_emotion_model.keras', monitor='val_loss', save_best_only=True)
]

In [ ]:
# Train the model
history = model.fit(
    x=X,
    y=y_emotion,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=callbacks
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X, y_emotion)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Evaluate the model on the validation set used during training
val_loss, val_accuracy = model.evaluate(X[int(0.8*len(X)):], y_emotion[int(0.8*len(y_emotion)):])
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

In [ ]:
# Extract loss values
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Create epochs range
epochs = range(1, len(training_loss) + 1)

# Plotting the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, 'r', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Example of predicting emotion from a specific image
image_index = 100  # Just an example index
print("Original Emotion: ", emotion_dict[df['emotion'][image_index]])

# Predict from model
pred = model.predict(X[image_index].reshape(1, 128, 128, 1))
pred_emotion = emotion_dict[np.argmax(pred)]

print("Predicted Emotion: ", pred_emotion)

plt.axis('off')
plt.imshow(X[image_index].reshape(128, 128), cmap='gray');
plt.title(f"Predicted Emotion: {pred_emotion}")
plt.show()